In [47]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [48]:
# Webscrape from url
url = 'https://ev-database.org/compare/cheapest-electric-vehicle#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-eff:prev~next=100~300|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all'
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [49]:
#Initially find all the div that has class list-item (where the data lies)
div = soup.find_all('div',attrs={'class':'list-item'})

In [50]:
#For acquiring Brand-Model values
title = [i.find('div',attrs={'class':'title-wrap'}) for i in div] #get the container including Brand-Model Text
a = [i.find('a') for i in title] # which lies within <a><a>
span = [i.find_all('span') for i in a] #Retrieve all span inside <a><a>

In [51]:
brand = [] #name of brand
model = [] #name of model
for i, j in span:
    brand.append(i.text) #the first index of span is Brand
    model.append(j.text) # second index is Model

In [52]:
battery = [i.find('div',{'title-wrap'}) for i in div]

In [53]:
battery_ = [i.find('span',{'battery'}) for i in battery]

In [54]:
battery_1 = [i.text for i in battery_]

In [55]:
# For pulling out Spec information
spec = [i.find('div', {'class': 'specs'}) for i in div] #choose spec containing div
col = [i.find_all('span',attrs={'tag'}) for i in spec][0] #tag includes the title for each spec info
columns_list = [i.text for i in col] # list of Spec Names

In [56]:
# get the actual values for spec
sp = [i.find_all('span') for i in spec]

#empty lists for appending the info 
zero_hund = []
top_speed = []
range_ = []
eff = []
fc = []

#looping each span and desginated index = values
for i in sp:
    zero_hund.append(i[1].text)
    top_speed.append(i[3].text)
    range_.append(i[5].text)
    eff.append(i[7].text)
    fc.append(i[9].text)

In [57]:
#price = [i.find('div',attrs={'class':'pricing align-right'}) for i in div]
price = soup.find_all('div',attrs={'class':'pricing align-right'})
price_ = [i.find('span',{'class':'country_de'}) for i in price]

In [58]:
german_p = [i.text for i in price_]

In [59]:
# empty dataframe with columns
v_spec = pd.DataFrame(columns = ['Brand', 'Model','Battery']+columns_list+['Price(German)'])

In [60]:
#updating the Dataframe with the information extracted
v_spec['Brand'] = brand
v_spec['Model'] = model
v_spec['0 - 100'] = zero_hund
v_spec['Top Speed'] = top_speed
v_spec['Range'] = range_
v_spec['Efficiency'] = eff
v_spec['Fastcharge'] = fc
v_spec['Price(German)'] = german_p
v_spec['Battery'] = battery_1

In [61]:
# Not in proper format for ML 
v_spec.head()

,Brand,Model,Battery,0 - 100,Top Speed,Range,Efficiency,Fastcharge,Price(German)
0,Renault,Kangoo Maxi ZE 33,31,22.4 sec,130 km/h,160 km,194 Wh/km,-,N/A
1,Nissan,Leaf,37,7.9 sec,144 km/h,225 km,164 Wh/km,230 km/h,"€29,990"
2,Audi,e-tron Sportback 55 quattro,86.5,5.7 sec,200 km/h,375 km,231 Wh/km,600 km/h,N/A
3,Porsche,Taycan Turbo S,83.7,2.8 sec,260 km/h,390 km,215 Wh/km,860 km/h,"€186,336"
4,Nissan,e-NV200 Evalia,37,14.0 sec,123 km/h,170 km,218 Wh/km,170 km/h,"€43,730"


In [62]:
#Preprocessing
v_spec['0 - 100'] = v_spec.apply(lambda x: x['0 - 100'].replace('sec',''),axis=1).astype(float)
v_spec['Top Speed'] = v_spec.apply(lambda x: x['Top Speed'].replace('km/h',''),axis=1).astype(int)
v_spec['Range'] = v_spec.apply(lambda x: x['Range'].replace('km',''),axis=1).astype(int).astype(int)
v_spec['Efficiency'] = v_spec.apply(lambda x: x['Efficiency'].replace('Wh/km',''),axis=1).astype(int)
v_spec['Battery'] = v_spec['Battery'].astype(float)

v_spec['Fastcharge'] = v_spec.apply(lambda x: x['Fastcharge'].replace('km/h',''),axis=1)
v_spec['Fastcharge'] = v_spec.apply(lambda x: x['Fastcharge'].replace('-',''),axis=1)
v_spec['Fastcharge'] = v_spec.apply(lambda x: x['Fastcharge'].replace(' ',''),axis=1)


v_spec['Price(German)'] = v_spec.apply(lambda x: x['Price(German)'].replace('€',''),axis=1)
v_spec['Price(German)'] = v_spec.apply(lambda x: x['Price(German)'].replace('N/A',''),axis=1)
v_spec['Price(German)'] = v_spec.apply(lambda x: x['Price(German)'].replace(',',''),axis=1)
v_spec['Price(German)'] = v_spec.apply(lambda x: x['Price(German)'].replace('-',''),axis=1)
v_spec['Price(German)'] = v_spec.apply(lambda x: x['Price(German)'].replace('*',''),axis=1)

In [63]:
v_spec = v_spec.replace(r'^\s*$', np.nan, regex=True)

In [64]:
v_spec['Fastcharge'].fillna(0,inplace = True)
v_spec['Price(German)'].fillna(0,inplace = True)

In [65]:
v_spec['Fastcharge'] = v_spec['Fastcharge'].astype(int)
v_spec['Price(German)'] = v_spec['Price(German)'].astype(int)

In [66]:
v_spec.replace(0,np.nan,inplace=True)

In [67]:
v_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Brand          190 non-null    object 
 1   Model          190 non-null    object 
 2   Battery        190 non-null    float64
 3   0 - 100        190 non-null    float64
 4   Top Speed      190 non-null    int32  
 5   Range          190 non-null    int32  
 6   Efficiency     190 non-null    int32  
 7   Fastcharge     185 non-null    float64
 8   Price(German)  176 non-null    float64
dtypes: float64(4), int32(3), object(2)
memory usage: 11.3+ KB


In [68]:
v_spec.dropna(inplace=True)

In [69]:
v_spec.reset_index(drop=True,inplace=True)

In [70]:
v_spec

,Brand,Model,Battery,0 - 100,Top Speed,Range,Efficiency,Fastcharge,Price(German)
0,Nissan,Leaf,37.0,7.9,144,225,164,230.0,29990.0
1,Porsche,Taycan Turbo S,83.7,2.8,260,390,215,860.0,186336.0
2,Nissan,e-NV200 Evalia,37.0,14.0,123,170,218,170.0,43730.0
3,Nissan,Leaf e+,56.0,7.3,157,325,172,390.0,37550.0
4,BMW,i3 120 Ah,37.9,7.3,150,235,161,270.0,39000.0
...,...,...,...,...,...,...,...,...,...
167,Peugeot,e-208,45.0,8.1,150,285,158,460.0,31950.0
168,Peugeot,e-2008 SUV,45.0,8.5,150,255,176,410.0,35450.0
169,Opel,Corsa-e,45.0,8.1,150,285,158,460.0,29900.0
170,Opel,Mokka-e,45.0,9.2,150,255,176,410.0,34110.0


In [71]:
v_spec.to_csv('vehicle_spec')